In [1]:
import jieba
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
import random

# 文本处理，将label和content分开

In [2]:
# preprocess用于将一个文本文档进行切词，并以字符串形式输出切词结果
path = './cnews.test.txt'
with open(path,'r',encoding='UTF-8') as f:
    cnews_test = f.readlines()
# 取test中前3000出来分为2000为训练样本，1000测试样本
cnews_test = cnews_test[500:1000]+cnews_test[1500:2000]+cnews_test[2500:3000]+cnews_test[3500:4000]+cnews_test[4500:5000]+cnews_test[5500:6000]
# 将test中的label取出
test_label,test_x = [],[]
n = list(range(len(cnews_test)))
random.shuffle(n)
for i in n:
    each = cnews_test[i]
    each0 = each.split('\t')
    test_label.append(each0[0])
    test_x.append(each0[1])

对文本内容进行分词后并以" "连接

In [3]:
# 取test中前3000出来分为2000为训练样本，1000测试样本
import jieba
# 使用jieba精确分词
test_x = [[each0 for each0 in jieba.cut(each)] for each in test_x]
test_x = [' '.join(each) for each in test_x]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lejon\AppData\Local\Temp\jieba.cache
Loading model cost 0.630 seconds.
Prefix dict has been built succesfully.


将3000个样本分为2500的train数据，500的test数据

In [4]:
train_X = test_x[:2500]
train_y = test_label[:2500]
test_X = test_x[2500:]
test_y = test_label[2500:]

多项式模型计算文本分类

In [5]:
count_vector = CountVectorizer()
# 该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j]表示j词在i类文本下的词频
vector_matrix = count_vector.fit_transform(train_X)

# tfidf度量模型
train_tfidf = TfidfTransformer(use_idf=False).fit_transform(vector_matrix)
# 将词频矩阵转化为权重矩阵，每一个特征值就是一个单词的TF-IDF值

# 调用MultinomialNB分类器进行训练
clf = MultinomialNB().fit(train_tfidf,train_y)

# 测试
test_vector = count_vector.transform(test_X)
test_tfidf = TfidfTransformer(use_idf=False).fit_transform(test_vector)
predict_result = clf.predict(test_tfidf)

以正确分类的个数，简单评测模型预测结果的准确率

In [6]:
# 评测预测效果
def accuracy_(test_y,predict):
    TP,num = 0,len(test_y)
    for i in range(num):
        if test_y[i]==predict[i]:
            TP+=1
    return TP/num

多项式模型分类效果

In [7]:
print('多项式模型分类效果：%f'%accuracy_(test_y,predict_result))

多项式模型分类效果：0.970000


伯努利模型计算文本分类

In [8]:
count_vector = CountVectorizer()
# 该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j]表示j词在i类文本下的词频
vector_matrix = count_vector.fit_transform(train_X)

# tfidf度量模型
train_tfidf = TfidfTransformer(use_idf=False).fit_transform(vector_matrix)
# 将词频矩阵转化为权重矩阵，每一个特征值就是一个单词的TF-IDF值

# 调用MultinomialNB分类器进行训练
clf = BernoulliNB().fit(train_tfidf,train_y)

# 测试
test_vector = count_vector.transform(test_X)
test_tfidf = TfidfTransformer(use_idf=False).fit_transform(test_vector)
predict_result = clf.predict(test_tfidf)

伯努利模型分类效果

In [9]:
print('伯努利模型分类效果：%f'%accuracy_(test_y,predict_result))

伯努利模型分类效果：0.870000


SVM进行文本分类

In [11]:
from sklearn import svm
count_vector = CountVectorizer()
# 该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j]表示j词在i类文本下的词频
vector_matrix = count_vector.fit_transform(train_X)

# tfidf度量模型
train_tfidf = TfidfTransformer(use_idf=False).fit_transform(vector_matrix)
# 将词频矩阵转化为权重矩阵，每一个特征值就是一个单词的TF-IDF值

# SVM分类
clf = svm.LinearSVC(C=1.6, class_weight=None,dual=True, fit_intercept=True,intercept_scaling=1, loss='squared_hinge',
                        max_iter=2500,multi_class='ovr', penalty='l2', random_state=None,tol=0.0001,verbose=0)
clf.fit(train_tfidf,train_y)

LinearSVC(C=1.6, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=2500,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [12]:
# SVM分类测试
test_vector = count_vector.transform(test_X)
test_tfidf = TfidfTransformer(use_idf=False).fit_transform(test_vector)
predict_result = clf.predict(test_tfidf)
print('SVM分类效果：%f'%accuracy_(test_y,predict_result))

SVM分类效果：0.986000


# LDA部分
获取训练矩阵和单词

In [38]:
from nltk.stem.wordnet import WordNetLemmatizer
# 数据准备
# preprocess用于将一个文本文档进行切词，并以字符串形式输出切词结果
path = './cnews.test.txt'
with open(path,'r',encoding='UTF-8') as f:
    cnews_test = f.readlines()
# 取test中前3000出来分为2000为训练样本，1000测试样本
cnews_test = cnews_test[500:1500]
# 将test中的label取出
test_label,test_x = [],[]
n = list(range(len(cnews_test)))
for i in n:
    each = cnews_test[i]
    each0 = each.split('\t')
    test_label.append(each0[0])
    test_x.append(each0[1])
    
# 载入停用词字典，对其进行去停用词
with open('./stopword.txt','r',encoding='UTF-8') as f:
    stopwords = f.readlines()
a = ''
for each in stopwords:
    a = a + ' '+each
stopwords = a.replace('\n','').split(' ')
stopwords = [each for each in stopwords if each not in ['\n']]

test_x = [[each0 for each0 in jieba.cut(each) if each0 not in stopwords] for each in test_x] 

In [28]:
test_x[0] # 查看文本数据

['科比',
 '骨折',
 ' ',
 '扣篮',
 '大伙',
 '!',
 '新浪',
 '体育讯',
 '北京',
 '时间',
 '27',
 '洛杉矶',
 '时间',
 '26',
 '消息',
 '洛杉矶',
 '湖人',
 '主场',
 '迎战',
 '新奥尔良',
 '黄蜂',
 '科比',
 '布莱恩特',
 '本场',
 '比赛',
 '带伤',
 '出战',
 '13',
 '投',
 '拿下',
 '19',
 '带领',
 '湖人',
 '106',
 '90',
 '胜利',
 '赛后',
 '发布厅',
 '接受',
 '采访',
 '赢球',
 '科比',
 '开心',
 '脸上',
 '挂',
 '笑容',
 '还',
 '不停',
 '记者',
 '开玩笑',
 '谈到',
 '脚踝',
 '情况',
 '科比',
 '脚踝',
 '僵硬',
 '打打球',
 '好',
 '越',
 '越',
 '放松',
 '最初',
 '几分钟',
 '好',
 '记者',
 '赛后',
 '兰德',
 '科比',
 '带伤',
 '打出',
 '精彩',
 '表现',
 '评价',
 '道',
 '受伤',
 '受伤',
 '科比',
 '笑',
 '结构',
 '人类',
 '科比',
 '解释',
 '拒绝',
 'MRI',
 '原因',
 '骨折',
 '折',
 '检查',
 'MRI',
 '浪费',
 '路上',
 '小时',
 '时间',
 '还',
 '关心',
 'MRI',
 '科比',
 '扣篮',
 '寓意',
 '这向',
 '传达',
 '信息',
 '科比',
 '两个',
 '扣篮',
 '精彩',
 '程度',
 '堪比',
 '扣篮',
 '大赛',
 '冠军',
 '表演',
 '问道',
 '想起',
 '23',
 '24',
 '岁',
 '科比',
 '科比',
 '确实',
 '想起',
 '年轻',
 '时代',
 '确实',
 '感觉',
 '82',
 '场',
 '比赛',
 '感觉',
 '确实',
 '阿里',
 '扎',
 '17',
 '投',
 '谈到',
 '这位',
 '夺冠',
 '队友',
 '科比',
 '左拳',
 '捶',
 '捶',
 '胸膛',
 '言

In [41]:
set(test_label)

{'体育', '娱乐'}

准备Documnet-Term矩阵

In [26]:
import gensim
from gensim import corpora

# 创建语料的词语词典，每个单独的词语都会被赋予一个索引
dictionary = corpora.Dictionary(test_x)

# 使用上面的词典，将转换文档列表（语料）变成 DT 矩阵
doc_term_matrix = [dictionary.doc2bow(doc) for doc in test_x]

构建LDA模型

In [37]:
#  使用 gensim 来创建 LDA 模型对象
Lda = gensim.models.ldamodel.LdaModel
# 在 DT 矩阵上运行和训练 LDA 模型
ldamodel = Lda(doc_term_matrix,num_topics=1,id2word=dictionary, passes=50)
# 输出结果
result_lda = ldamodel.print_topics(num_topics=1,num_words=5)
print(result_lda)

[(0, '0.000*"冯绍峰" + 0.000*"霸气" + 0.000*"项羽" + 0.000*"鸿门宴" + 0.000*"刘亦菲"')]


想要得到每一个文本对应的主题，因此对

In [40]:
# 创建语料的词语词典，每个单独的词语都会被赋予一个索引
dictionary = corpora.Dictionary(test_x)
# 使用上面的词典，将转换文档列表（语料）变成 DT 矩阵
doc_term_matrix = [dictionary.doc2bow(doc) for doc in test_x]

#  使用 gensim 来创建 LDA 模型对象
Lda = gensim.models.ldamodel.LdaModel
# 在 DT 矩阵上运行和训练 LDA 模型
ldamodel = Lda(doc_term_matrix,num_topics=2,id2word=dictionary, passes=50)
# 输出结果
result_lda = ldamodel.print_topics(num_topics=2,num_words=20)
print(result_lda)

[(0, '0.010*" " + 0.010*"微博" + 0.008*"导演" + 0.006*"影片" + 0.005*"观众" + 0.004*"还" + 0.004*"新浪" + 0.004*"票房" + 0.004*"娱乐" + 0.003*"拍" + 0.003*"拍摄" + 0.003*"中国" + 0.003*"好" + 0.003*"\xa0" + 0.003*"角色" + 0.003*"演员" + 0.003*"\n" + 0.003*"时" + 0.002*"戏" + 0.002*"主演"'), (1, '0.010*"比赛" + 0.007*"球队" + 0.006*" " + 0.005*"季后赛" + 0.005*"篮板" + 0.005*"球员" + 0.004*"还" + 0.004*"次" + 0.004*"时间" + 0.004*"新浪" + 0.004*"湖人" + 0.004*"赛季" + 0.004*"热火" + 0.004*"防守" + 0.004*"时" + 0.004*"进攻" + 0.004*"科比" + 0.003*"\n" + 0.003*"马刺" + 0.003*"三分"')]


In [47]:
import re
result_Lda = {}
for each in result_lda:
    each0 = each[1].split('+')
    for each1 in each0:
        each2 = each1.split('*')
        if each2[1] in result_Lda:
            result_Lda[each2[1]] += int(re.sub("\D","",each2[0]))
        else:
            result_Lda[each2[1]] = int(re.sub("\D","",each2[0]))

In [48]:
result_Lda

{'" " ': 16,
 '"微博" ': 10,
 '"导演" ': 8,
 '"影片" ': 6,
 '"观众" ': 5,
 '"还" ': 8,
 '"新浪" ': 8,
 '"票房" ': 4,
 '"娱乐" ': 4,
 '"拍" ': 3,
 '"拍摄" ': 3,
 '"中国" ': 3,
 '"好" ': 3,
 '"\xa0" ': 3,
 '"角色" ': 3,
 '"演员" ': 3,
 '"\n" ': 6,
 '"时" ': 7,
 '"戏" ': 2,
 '"主演"': 2,
 '"比赛" ': 10,
 '"球队" ': 7,
 '"季后赛" ': 5,
 '"篮板" ': 5,
 '"球员" ': 5,
 '"次" ': 4,
 '"时间" ': 4,
 '"湖人" ': 4,
 '"赛季" ': 4,
 '"热火" ': 4,
 '"防守" ': 4,
 '"进攻" ': 4,
 '"科比" ': 4,
 '"马刺" ': 3,
 '"三分"': 3}